# 03 - Technical Indicators
Compute rolling technical indicators (SMA, RSI, MACD) directly inside the notebook so no external module is required.

In [ ]:
# !pip install -r ../requirements.txt
from pathlib import Path
import sys
import pandas as pd

repo_root = Path.cwd().resolve().parent
if repo_root.name == 'notebooks':
    repo_root = repo_root.parent
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

from src.data_loader import load_price_csv

def add_moving_average(df: pd.DataFrame, window: int = 20, price_col: str = 'Close') -> pd.DataFrame:
    df = df.copy()
    df[f'MA_{window}'] = df[price_col].rolling(window=window, min_periods=window).mean()
    return df

def add_rsi(df: pd.DataFrame, window: int = 14, price_col: str = 'Close') -> pd.DataFrame:
    df = df.copy()
    delta = df[price_col].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=window, min_periods=window).mean()
    avg_loss = loss.rolling(window=window, min_periods=window).mean()
    rs = avg_gain / avg_loss.replace(0, pd.NA)
    df[f'RSI_{window}'] = 100 - (100 / (1 + rs))
    return df

def add_macd(df: pd.DataFrame, fast: int = 12, slow: int = 26, signal: int = 9, price_col: str = 'Close') -> pd.DataFrame:
    df = df.copy()
    ema_fast = df[price_col].ewm(span=fast, adjust=False).mean()
    ema_slow = df[price_col].ewm(span=slow, adjust=False).mean()
    df['MACD'] = ema_fast - ema_slow
    df['MACD_signal'] = df['MACD'].ewm(span=signal, adjust=False).mean()
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']
    return df

price_path = repo_root / 'data' / 'AAPL.csv'
price = load_price_csv(price_path)
price = add_moving_average(price, window=20)
price = add_rsi(price, window=14)
price = add_macd(price)
display(price.tail())

           Date       Close        High         Low        Open    Volume  \
3769 2023-12-22  191.788757  193.581821  191.164647  193.353962  37149600   
3770 2023-12-26  191.243912  192.076049  191.025969  191.798670  28919300   
3771 2023-12-27  191.342972  191.689703  189.302247  190.689158  48087700   
3772 2023-12-28  191.768951  192.838849  191.362784  192.323710  34049900   
3773 2023-12-29  190.728775  192.581275  189.936256  192.085953  42672100   

           MA_20     RSI_14      MACD  MACD_signal  MACD_hist  
3769  191.856618  59.246127  2.633141     3.235256  -0.602115  
3770  192.018094  49.031899  2.304751     3.049155  -0.744404  
3771  192.154308  52.291510  2.029104     2.845145  -0.816041  
3772  192.362839  47.920425  1.823998     2.640915  -0.816918  
3773  192.490633  40.185234  1.559539     2.424640  -0.865101  


_Set `price_path` to any of the other CSVs in `data/` (AMZN, GOOG, NVDA, etc.) to reuse the same indicator pipeline._